# Chapter 2: Tests
In this chapter, we explain the main idea behind `InteractiveTest`, three example interactive test classes provided by ldmunit and how you can implement your own interactive tests with little effort.

## Interactive Tests
`InteractiveTest` class is one implementation of a test class which **requires** the models to implement `ldmunit.capabilities.Interactive` interface. Such models should implement the methods that allow them to mix predictions and model updates. This is in contrast to models that first must be trained, and then can only predict actions for stimuli with no further training.

In [2]:
from ldmunit.tests import InteractiveTest
# InteractiveTest documentation
print(InteractiveTest.__doc__)


    Perform interactive tests by feeding the input samples (stimuli) one at a
    time. This class is not intended to be used directly since it does not
    specify how the score should be computed. In order to create concrete
    interactive tests, create a subclass and specify how the score should be
    computed.

    See Also
    --------
    :class:`NLLTest`, :class:`AICTest`, :class:`BICTest` for examples of concrete interactive test classes
    


In [3]:
# models are required to implement ldmunit.capabilities.Interactive
InteractiveTest.required_capabilities

(ldmunit.capabilities.Interactive,)

`InteractiveTest` class overrides the `generate_prediction` method of `sciunit.Test` class to test a given multi-subject model interactively. It requires three sets of observations:
* stimuli
* actions
* rewards

However, `InteractiveTest` class does not specify a complete test class because it has no notion of score. It simply defines a way of generating the predictions. In order to create a real, interactive test, we need to specify how the score is computed

## Examples Provided by ldmunit
ldmunit provides three concrete interactive tests classes that inherit from `InteractiveTest`: `NLLTest`, `AICTest` and `BICTest`.

We have seen `NLLTest` in the previous chapter: it uses negative log-likelihood as score to compare models. In order to compute negative log-likelihood from a set of predictions, `NLLTest` requires probability distributions for each input stimulus. This is a new requirement which we can see:

In [6]:
from ldmunit.tests import NLLTest
NLLTest.required_capabilities

(ldmunit.capabilities.Interactive, ldmunit.capabilities.LogProbModel)

and the description for `LogProbModel` states that

In [8]:
from ldmunit.capabilities import LogProbModel
print(LogProbModel.__doc__)


    Capability for models that produce a log probability distribution
    as a result of their predict function.

    Models with this capability are required to have the following methods
    


As we mentioned before, the other example tests provided are `AICTest` which computes the score using Akaike's Information Criterion and `BICTest` which uses Bayesian Information Criterion as score. These scores also require probability distributions for each stimuli and their implementations are similar in spirit to `NLLTest`.

Different from `NLLTest`, `AICTest` and `BICTest` require information about the properties of the model being tested. This information is not available in `compute_score` method where all the predictions are already generated. For this reason, they intercept `generate_prediction` call to collect information about the model before utilizing the already implemented prediction generation logic from `InteractiveTest`.

## Implementing Your Own Interactive Tests
Implementations of the three example test classes provided by ldmunit are good starting points for implementing your own interactive tests. You can tweak various parts of the new classes such as scores to use, how it is computed and even alter the way predictions are generated by partially or completely overriding `generate_prediction` method. Further, new capabilities can be added as in all three of provided examples.